<a href="https://colab.research.google.com/github/BaekKyunShin/Kaggle-Competition-Including-Dacon/blob/master/Novel_Writer_Classification/baseline_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Baseline Code

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords 
import re
import nltk

In [ ]:
!ls

adc.json  gdrive  sample_data


In [ ]:
#경로 설정
import os
os.chdir('/content/gdrive/My Drive/colab/Dacon_Novel_Writer_Classification/')

In [ ]:
#파일 불러오기
train = pd.read_csv('open/train.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [ ]:
train.head()

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [ ]:
test.head()

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...


In [ ]:
sample_submission.head()

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0


# 전처리

In [ ]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

train.head()

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3


In [ ]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in final_stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
basic_stopwords = { "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" }

nltk.download('stopwords')
nltk_stopwords = set(stopwords.words('english'))

final_stopwords = nltk_stopwords.union(basic_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [ ]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [ ]:
X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere wellwisher friend sister lucy odin',
       'wanted lend money', 'certainly occurred said yes like'],
      dtype='<U1405')

In [ ]:
len(X_train)

54879

In [ ]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
oov_tok = "<OOV>"

In [ ]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words=vocab_size) #, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

word_index

{'odin': 1,
 'said': 2,
 'one': 3,
 'mr': 4,
 'upon': 5,
 'man': 6,
 'know': 7,
 'well': 8,
 'time': 9,
 'little': 10,
 'see': 11,
 'come': 12,
 'like': 13,
 'must': 14,
 'odins': 15,
 'much': 16,
 'say': 17,
 'think': 18,
 'us': 19,
 'never': 20,
 'good': 21,
 'cried': 22,
 'may': 23,
 'sir': 24,
 'go': 25,
 'dont': 26,
 'two': 27,
 'made': 28,
 'nothing': 29,
 'though': 30,
 'came': 31,
 'old': 32,
 'last': 33,
 'great': 34,
 'thought': 35,
 'might': 36,
 'shall': 37,
 'way': 38,
 'first': 39,
 'hand': 40,
 'back': 41,
 'even': 42,
 'face': 43,
 'long': 44,
 'away': 45,
 'mrs': 46,
 'without': 47,
 'every': 48,
 'miss': 49,
 'still': 50,
 'eyes': 51,
 'looked': 52,
 'asked': 53,
 'day': 54,
 'house': 55,
 'went': 56,
 'tell': 57,
 'make': 58,
 'young': 59,
 'take': 60,
 'quite': 61,
 'room': 62,
 'something': 63,
 'head': 64,
 'look': 65,
 'dear': 66,
 'yet': 67,
 'yes': 68,
 'another': 69,
 'ever': 70,
 'let': 71,
 'saw': 72,
 'mind': 73,
 'life': 74,
 'door': 75,
 'oh': 76,
 'heard

In [ ]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [ ]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 16)           320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# fit model
num_epochs = 30
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/30
1372/1372 - 7s - loss: 1.5666 - accuracy: 0.2762 - val_loss: 1.5422 - val_accuracy: 0.2757
Epoch 2/30
1372/1372 - 7s - loss: 1.4233 - accuracy: 0.3941 - val_loss: 1.2970 - val_accuracy: 0.4958
Epoch 3/30
1372/1372 - 7s - loss: 1.1891 - accuracy: 0.5311 - val_loss: 1.1250 - val_accuracy: 0.5549
Epoch 4/30
1372/1372 - 7s - loss: 1.0479 - accuracy: 0.5764 - val_loss: 1.0799 - val_accuracy: 0.5441
Epoch 5/30
1372/1372 - 7s - loss: 0.9618 - accuracy: 0.6101 - val_loss: 0.9919 - val_accuracy: 0.5967
Epoch 6/30
1372/1372 - 7s - loss: 0.8974 - accuracy: 0.6400 - val_loss: 0.9470 - val_accuracy: 0.6220
Epoch 7/30
1372/1372 - 7s - loss: 0.8414 - accuracy: 0.6670 - val_loss: 0.9027 - val_accuracy: 0.6463
Epoch 8/30
1372/1372 - 7s - loss: 0.7862 - accuracy: 0.6949 - val_loss: 0.8694 - val_accuracy: 0.6586
Epoch 9/30
1372/1372 - 7s - loss: 0.7348 - accuracy: 0.7193 - val_loss: 0.8467 - val_accuracy: 0.6729
Epoch 10/30
1372/1372 - 7s - loss: 0.6914 - accuracy: 0.7402 - val_loss: 0.8414 - 

In [ ]:
# predict values
pred = model.predict_proba(test_padded)

Instructions for updating:
Please use `model.predict()` instead.


In [ ]:
pred

array([[3.2764765e-05, 1.4217783e-02, 3.4449466e-02, 9.5057881e-01,
        7.2113221e-04],
       [3.3102721e-02, 6.9222039e-01, 3.7391558e-02, 1.8746254e-01,
        4.9822778e-02],
       [9.9999344e-01, 6.5900499e-06, 2.7577184e-11, 2.0278424e-13,
        6.6980839e-09],
       ...,
       [7.1432667e-05, 9.9992800e-01, 4.6329321e-12, 5.7045446e-07,
        2.4518496e-09],
       [5.3678792e-05, 9.9994588e-01, 2.2589916e-10, 2.9683679e-07,
        2.7520886e-07],
       [9.9995232e-01, 4.2508182e-05, 4.9624822e-07, 4.3215769e-09,
        4.6299842e-06]], dtype=float32)

In [ ]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,3.276476e-05,1.421778e-02,3.444947e-02,9.505788e-01,7.211322e-04
1,1,3.310272e-02,6.922204e-01,3.739156e-02,1.874625e-01,4.982278e-02
2,2,9.999934e-01,6.590050e-06,2.757718e-11,2.027842e-13,6.698084e-09
3,3,1.455378e-07,2.692983e-13,9.999945e-01,1.248287e-09,5.353179e-06
4,4,9.826592e-01,1.299240e-04,3.199780e-03,1.400718e-02,3.958221e-06
...,...,...,...,...,...,...
19612,19612,9.421856e-08,9.999999e-01,1.348828e-23,1.416498e-12,8.179792e-20
19613,19613,5.330207e-03,3.187120e-05,9.051159e-06,9.598511e-17,9.946288e-01
19614,19614,7.143267e-05,9.999280e-01,4.632932e-12,5.704545e-07,2.451850e-09
19615,19615,5.367879e-05,9.999459e-01,2.258992e-10,2.968368e-07,2.752089e-07


In [ ]:
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')